In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re
import mysql.connector

# ESTRUCTURA DE BBDD EN MYSQL


    create database factoria_jamones;
    use factoria_jamones;

    CREATE TABLE `raza` (
      `id_raza` integer,
      `desc_raza` varchar(100),
      PRIMARY KEY (`id_raza`)
    );

    CREATE TABLE `matadero` (
      `id_matadero` integer,
      `desc_matadero` varchar(100),
      PRIMARY KEY (`id_matadero`)
    );

    CREATE TABLE `alimentacion` (
      `id_alimentacion` integer,
      `desc_alimentacion` varchar(100),
      PRIMARY KEY (`id_alimentacion`)
    );

    CREATE TABLE `tipo_pesada_entrada` (
      `id_tipo_pesada_entrada` integer,
      `desc_tipo_pesada_entrada` varchar(100),
      PRIMARY KEY (`id_tipo_pesada_entrada`)
    );

    CREATE TABLE `partida` (
      `id_partida` integer,
      `id_raza` integer,
      `añada` integer,
      `acido_oleico` float,
      `acido_palmitico` float,
      `acido_estearico` float,
      `acido_linoleico` float,
      `peso_varios` float,
      `peso_vientres` float,
      `peso_cerdos` float,
      `peso_neto_canal` float,
      `no_animales` integer,
      `nacimiento` date,
      `id_matadero` integer,
      `id_alimentacion` integer,
      `id_tipo_pesada_entrada` integer,
      `fecha_entrada` date,
      PRIMARY KEY (`id_partida`),
      FOREIGN KEY (`id_raza`) REFERENCES `raza`(`id_raza`),
      FOREIGN KEY (`id_matadero`) REFERENCES `matadero`(`id_matadero`),
      FOREIGN KEY (`id_tipo_pesada_entrada`) REFERENCES `tipo_pesada_entrada`(`id_tipo_pesada_entrada`),
      FOREIGN KEY (`id_alimentacion`) REFERENCES `alimentacion`(`id_alimentacion`)
    );



    CREATE TABLE `tipo_pesada_salida` (
      `id_tipo_pesada_salida` integer,
      `desc_tipo_pesada_salida` varchar(100),
      PRIMARY KEY (`id_tipo_pesada_salida`)
    );

    CREATE TABLE `estado_salida` (
      `id_estado_salida` integer,
      `desc_estado_salida` varchar(100),
      PRIMARY KEY (`id_estado_salida`)
    );

    CREATE TABLE `tipo_pieza` (
      `id_tipo_pieza` integer,
      `desc_tipo_pieza` varchar(100),
      PRIMARY KEY (`id_tipo_pieza`)
    );

    CREATE TABLE `pieza` (
      `id_pieza` integer auto_increment,
      `no_serie` varchar(100),
      `id_tipo_pieza` integer,
      `id_partida` integer,
      `peso_salida` float, 
      `fecha_salida` date,
      `id_tipo_pesada_salida` integer,
      `id_estado_salida` integer,
      `numero_serie_lc` integer,
      PRIMARY KEY (`id_pieza`),
      FOREIGN KEY (`id_tipo_pesada_salida`) REFERENCES `tipo_pesada_salida`(`id_tipo_pesada_salida`),
      FOREIGN KEY (`id_estado_salida`) REFERENCES `estado_salida`(`id_estado_salida`),
      FOREIGN KEY (`id_partida`) REFERENCES `partida`(`id_partida`),
      FOREIGN KEY (`id_tipo_pieza`) REFERENCES `tipo_pieza`(`id_tipo_pieza`)
    );


In [2]:
# Cargamos el STEP 1 trabajado en el script anterior

df_Base = pd.read_csv('Step1JamonesDataExtraction.csv',sep =";" ,na_values = "NULL", index_col=None)
df_Base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150434 entries, 0 to 150433
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   TipoArticuloEscandallo  150434 non-null  object 
 1   Añada                   150434 non-null  int64  
 2   Partida                 150434 non-null  int64  
 3   Raza                    150434 non-null  object 
 4   Alimentacion            150434 non-null  object 
 5   NumeroSerieLc           150434 non-null  int64  
 6   AcidoOleico             150434 non-null  float64
 7   AcidoPalmitico          150434 non-null  float64
 8   AcidoEstearico          150434 non-null  float64
 9   AcidoLinoleico          150434 non-null  float64
 10  PesoVarios              150434 non-null  float64
 11  Pesovientres            150434 non-null  float64
 12  PesoCerdos              150434 non-null  float64
 13  PesoNetoCanal           150434 non-null  float64
 14  Nacimiento          

# PASO 1 PREPARAR DATOS PARA INTRODUCCIÓN EN BBDD

 Las tablas las llenaremos en el mismo orden que se crearon en la base de la siguiente manera:

## .- Paso 1.1 Diccionarios de categoría: categoría-id
    .- Paso 1.1.1 Transformamos categorías en diccionarios
        raza
        matadero
        alimentacion
        tipo_pesada_salida
        tipo_pesada_entrada
        estado_salida
        tipo_pieza

    .- Paso 1.1.2 Sustituimos id por categoría en el dataframe con el que crearemos la siguiente tabla

## .- Paso 1.2 Tabla partida: previo checkeo de valores únicos, hay que crear la tabla en python mediante una agrupación groupby.

    .- Paso 1.2.1 Comprobar que los datos se agrupan por categorías y corregir aquellos que no lo hagan
    
    .- Paso 1.2.2 Crear la tabla de partida
    
    .- Paso 1.2.3 Sustituimos id por partida en el dataframe con el que crearemos la siguiente tabla
        
## .- Paso 1.3 Tabla pieza

### .- Paso 1.1 Tablas de categoría se introduciran mediante una diccionario creado aplicada a la columna que corresponda con el método unique():

    raza
    matadero
    alimentacion
    tipo_pesada_salida
    tipo_pesada_entrada
    estado_salida
    tipo_pieza

#### .- Paso 1.1.1 Transformamos categorías en diccionarios

In [3]:
# Funcion a la cual le introducimos una lista de categorías y nos crea un diccionario clave-valor donde las claves 
# serán las ids de las categorías

def DiccCats(listadoCategorias):

    categorias = {}
    n = 0
    for values in listadoCategorias:
        categorias[values] = n
        n+=1
    return categorias

In [4]:
# Cuando necesitemos los diccionarios introduciremos directamente la función con la categoría de la siguiente manera para
# ahorrar variables

print(DiccCats(df_Base["Raza"].unique()))
print(DiccCats(df_Base["CodigoMatadero"].unique().astype("object")))
print(DiccCats(df_Base["Alimentacion"].unique()))
print(DiccCats(df_Base["TipoPesadaSalida"].unique()))
print(DiccCats(df_Base["TipoPesada"].unique()))
print(DiccCats(df_Base["EstadoProductoSalida"].unique()))
print(DiccCats(df_Base["TipoArticuloEscandallo"].unique()))

{'50% IBERICO': 0, '75% IBERICO': 1, '100% IBERICO': 2}
{1: 0, 0: 1, 3: 2, 2: 3}
{'BELLOTA': 0, 'CEBO DE CAMPO': 1, 'CEBO': 2}
{'Pesada PIEZA ENTERA': 0, 'Pesada pieza deshuesada': 1, 'Pesada blisters para caja': 2}
{'Grupal': 0, 'Individual': 1}
{'NO APTO PARA CONSUMO': 0, 'APTO PARA CONSUMO': 1}
{'Jamón': 0, 'Paleta': 1}


In [5]:
# Checkeamos que los valores de categorías son los correctos

print(df_Base["Raza"].unique())
print(df_Base["CodigoMatadero"].unique())
print(df_Base["Alimentacion"].unique())
print(df_Base["TipoPesadaSalida"].unique())
print(df_Base["EstadoProductoSalida"].unique())
print(df_Base["TipoPesada"].unique())
print(df_Base["TipoArticuloEscandallo"].unique())

['50% IBERICO' '75% IBERICO' '100% IBERICO']
[1 0 3 2]
['BELLOTA' 'CEBO DE CAMPO' 'CEBO']
['Pesada PIEZA ENTERA' 'Pesada pieza deshuesada'
 'Pesada blisters para caja']
['NO APTO PARA CONSUMO' 'APTO PARA CONSUMO']
['Grupal' 'Individual']
['Jamón' 'Paleta']


#### .- Paso 1.1.2 Sustituimos id por categoría en el dataset que usaremos para el resto de tablas

In [6]:
# Para introducir correctamente los datos en las tablas sustituimos 
# los valores de las categorías por sus ids en el dataset principal.

df_InDt1 = df_Base.copy()
lista = ["Raza","CodigoMatadero","Alimentacion","TipoPesadaSalida","TipoPesada","EstadoProductoSalida","TipoArticuloEscandallo"]

for columns in lista:
    df_InDt1[columns].replace(to_replace=DiccCats(df_InDt1[columns].unique()), inplace=True)
    

print(df_InDt1["Raza"].unique())
print(df_InDt1["CodigoMatadero"].unique())
print(df_InDt1["Alimentacion"].unique())
print(df_InDt1["TipoPesadaSalida"].unique())
print(df_InDt1["EstadoProductoSalida"].unique())
print(df_InDt1["TipoPesada"].unique())
print(df_InDt1["TipoArticuloEscandallo"].unique())


[0 1 2]
[0 1 2 3]
[0 1 2]
[0 1 2]
[0 1]
[0 1]
[0 1]


### .- Paso 1.2 Tabla partida: previo checkeo de valores únicos, hay que crear la tabla en python mediante una agrupación groupby.

#### .- Paso 1.2.1 Comprobar que los datos se agrupan por categorías y corregir aquellos que no lo hagan

    # Las columnas indicadas son las que son variables de la tabla partida (nunique para una partida)

    # TipoArticuloEscandallo      2
    # Añada                       1 [1]
    # Partida                     1 
    # Raza                        1 [3]
    # Alimentacion                1 [4]
    # NumeroSerieLc             156
    # AcidoOleico                 0 [6]
    # AcidoPalmitico              0 [7]
    # AcidoEstearico              0 [8]
    # AcidoLinoleico              0 [9]
    # PesoVarios                  1 [10]
    # Pesovientres                1 [11]
    # PesoCerdos                  1 [12]
    # PesoNetoCanal               1 [13]
    # Nacimiento                  1 [14]
    # CodigoMatadero              1 [15]
    # NroAnimales                 1 [16]
    # PesoEntradaEntrada          0
    # TipoPesada                  1 [18]
    # PesoSalida                122
    # DescripcionArticulo         8
    # FechaEntrada                1 [21]
    # FechaSalida                62
    # TipoPesadaSalida            1
    # EstadoProductoSalida        2
    # dtype: int64

In [7]:
#Función checkeo valores repetidos para valores por partida

def checkRepValPerPart(df):

    checkValues = [1,3,4,6,8,9,10,11,12,13,14,15,16,18,21]

    for partidas in df["Partida"].unique():
        for index in checkValues:
            if df[df["Partida"]==partidas].nunique()[index]==1 or df[df["Partida"]==partidas].nunique()[index]==0:
                continue
            else:
                print("Partidas "+ str(partidas) +" tiene repetidas en columna "+ str(index))

In [8]:


# checkRepValPerPart(df_InDt1)


In [9]:
# Comprobando repeticiones y preguntando al proveedor de los datos llegamos a la conclusion
# de que los números de partida no son únicos y por tanto los renombraremos mediante
# concatenación de año-mes/nºpartida
df_InDt2 = df_InDt1.copy()
df_InDt2['FechaEntrada'] =  pd.to_datetime(df_InDt2['FechaEntrada'])

df_InDt2["Partida"]= df_InDt2['FechaEntrada'].dt.strftime('%Y-%m').astype("str").map(str) + "/" + df_InDt2["Partida"].astype("str")
df_InDt2

,TipoArticuloEscandallo,Añada,Partida,Raza,Alimentacion,NumeroSerieLc,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,...,CodigoMatadero,NroAnimales,PesoEntradaEntrada,TipoPesada,PesoSalida,DescripcionArticulo,FechaEntrada,FechaSalida,TipoPesadaSalida,EstadoProductoSalida
0,0,2014,2014-01/102,0,0,140100102008,53.76,21.02,10.61,8.11,...,0,117,13.962100,0,9.400,JAMÓN NO APTO PARA EL CONSUMO HUMANO,2014-01-13,2018-04-24 00:00:00.000,0,0
1,0,2014,2014-01/102,0,0,140100102030,53.76,21.02,10.61,8.11,...,0,117,14.580588,0,10.400,JAMON DE BELLOTA IBERICO PASSION,2014-01-13,2018-06-04 00:00:00.000,0,1
2,1,2014,2014-01/102,0,0,141100102051,53.76,21.02,10.61,8.11,...,0,117,9.603106,0,5.964,PALETA DE BELLOTA IBÉRICA D.O.,2014-01-13,2016-05-06 00:00:00.000,0,1
3,1,2014,2014-01/102,0,0,141100102001,53.76,21.02,10.61,8.11,...,0,117,7.682340,0,3.150,PALETA DE BELLOTA IBÉRICA DESH.,2014-01-13,2016-12-01 00:00:00.000,1,1
4,1,2014,2014-01/102,0,0,141100102155,53.76,21.02,10.61,8.11,...,0,117,7.616813,0,3.054,PALETA DE BELLOTA IB. D.O. DESH.,2014-01-13,2016-08-01 00:00:00.000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150429,1,2018,2018-02/999,1,0,237030576292,0.00,0.00,0.00,0.00,...,0,65,10.149980,0,6.383,PALETA DE BELLOTA IBÉRICA,2018-02-12,2021-04-13 00:00:00.000,0,1
150430,1,2018,2018-02/999,1,0,237030576292,0.00,0.00,0.00,0.00,...,0,65,7.885824,0,3.160,PALETA DE BELLOTA IBÉRICA,2018-02-12,2021-04-13 00:00:00.000,0,1
150431,1,2018,2018-02/999,1,0,237030576292,0.00,0.00,0.00,0.00,...,0,65,10.149980,0,6.383,PALETA DE BELLOTA IBÉRICA DESH.,2018-02-12,2021-04-16 00:00:00.000,1,1
150432,1,2018,2018-02/999,1,0,237030576292,0.00,0.00,0.00,0.00,...,0,65,7.885824,0,3.160,PALETA DE BELLOTA IBÉRICA DESH.,2018-02-12,2021-04-16 00:00:00.000,1,1


In [10]:
# Volvemos a checkear y como podemos comprobar fallan fechas de entrada que se han metido incorrectamente porque 
# seguramente no hayan introducido los datos de toda la partida, el mismo día


# checkRepValPerPart(df_InDt2)

#### .- Paso 1.2.2 Crear la tabla de partida

In [11]:
# Creamos el dataframe con las columnas que corresponden a la tabla "partida"
checkValues = [1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,18,21]
colsPartidas = []
for pos in checkValues:
    colsPartidas.append(df_InDt2.columns[pos])

df_Ptda1 = df_InDt2.loc[:,colsPartidas]
df_Ptda1

,Añada,Partida,Raza,Alimentacion,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,PesoVarios,Pesovientres,PesoCerdos,PesoNetoCanal,Nacimiento,CodigoMatadero,NroAnimales,TipoPesada,FechaEntrada
0,2014,2014-01/102,0,0,53.76,21.02,10.61,8.11,9413.398,16685.1,20600.0,16351.398,NaN,0,117,0,2014-01-13
1,2014,2014-01/102,0,0,53.76,21.02,10.61,8.11,9413.398,16685.1,20600.0,16351.398,NaN,0,117,0,2014-01-13
2,2014,2014-01/102,0,0,53.76,21.02,10.61,8.11,9413.398,16685.1,20600.0,16351.398,NaN,0,117,0,2014-01-13
3,2014,2014-01/102,0,0,53.76,21.02,10.61,8.11,9413.398,16685.1,20600.0,16351.398,NaN,0,117,0,2014-01-13
4,2014,2014-01/102,0,0,53.76,21.02,10.61,8.11,9413.398,16685.1,20600.0,16351.398,NaN,0,117,0,2014-01-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150429,2018,2018-02/999,1,0,0.00,0.00,0.00,0.00,5244.962,9846.9,11596.0,9649.962,10/05/2016,0,65,0,2018-02-12
150430,2018,2018-02/999,1,0,0.00,0.00,0.00,0.00,5244.962,9846.9,11596.0,9649.962,10/05/2016,0,65,0,2018-02-12
150431,2018,2018-02/999,1,0,0.00,0.00,0.00,0.00,5244.962,9846.9,11596.0,9649.962,10/05/2016,0,65,0,2018-02-12
150432,2018,2018-02/999,1,0,0.00,0.00,0.00,0.00,5244.962,9846.9,11596.0,9649.962,10/05/2016,0,65,0,2018-02-12


In [12]:
# Para crear que funcione correctamente el groupby hay que sustituir los valores nan por un
# valor alternativo que devolveremos posteriormente a valor nan

df_Ptda1.isna().sum()

Añada                 0
Partida               0
Raza                  0
Alimentacion          0
AcidoOleico           0
AcidoPalmitico        0
AcidoEstearico        0
AcidoLinoleico        0
PesoVarios            0
Pesovientres          0
PesoCerdos            0
PesoNetoCanal         0
Nacimiento        29953
CodigoMatadero        0
NroAnimales           0
TipoPesada            0
FechaEntrada          0
dtype: int64

In [13]:
df_Ptda2 = df_Ptda1.copy()
df_Ptda2["AcidoOleico"] = df_Ptda2["AcidoOleico"].fillna(-1) 
df_Ptda2["AcidoPalmitico"] = df_Ptda2["AcidoPalmitico"].fillna(-1) 
df_Ptda2["AcidoEstearico"] = df_Ptda2["AcidoEstearico"].fillna(-1) 
df_Ptda2["AcidoLinoleico"] = df_Ptda2["AcidoLinoleico"].fillna(-1) 
df_Ptda2["PesoVarios"] = df_Ptda2["PesoVarios"].fillna(-1) 
df_Ptda2["Pesovientres"] = df_Ptda2["Pesovientres"].fillna(-1)
df_Ptda2["PesoCerdos"] = df_Ptda2["PesoCerdos"].fillna(-1)  
df_Ptda2["PesoNetoCanal"] = df_Ptda2["PesoNetoCanal"].fillna(-1)


In [14]:
# Para crear la tabla de Partida agrupo por partidas tomando la moda de sus valores 
# (comprobación previa de que las desviaciones son partidas que se han introducido en varios días)

df_Ptda3 = df_Ptda2.copy()
df_Ptda3= df_Ptda3.groupby("Partida").agg(lambda x: x.mode())


# Introducimos la id de partida

df_Ptda3 = df_Ptda3.reset_index()
df_Ptda3.insert(0,"IdPartida",df_Ptda3.index,True)

df_Ptda3["Nacimiento"] = df_Ptda3["Nacimiento"].astype("str")
df_Ptda3["FechaEntrada"] = df_Ptda3["FechaEntrada"].astype("str")

df_Ptda3

,IdPartida,Partida,Añada,Raza,Alimentacion,AcidoOleico,AcidoPalmitico,AcidoEstearico,AcidoLinoleico,PesoVarios,Pesovientres,PesoCerdos,PesoNetoCanal,Nacimiento,CodigoMatadero,NroAnimales,TipoPesada,FechaEntrada
0,0,2013-01/124,2013,0,0,55.50,20.76,8.19,8.40,7654.786,13145.7,16180.0,12882.786,20/05/2011,0,100,0,2013-01-14
1,1,2013-01/165,2013,0,2,0.00,0.00,0.00,0.00,7669.250,15212.5,18400.0,14908.250,10/01/2012,0,120,0,2013-01-16
2,2,2013-01/223,2013,0,0,53.66,20.60,10.75,9.16,4886.350,8657.5,10800.0,8484.350,15/09/2011,0,65,0,2013-01-21
3,3,2013-01/298,2013,0,2,0.00,0.00,0.00,0.00,8519.144,17182.8,20700.0,16839.144,10/01/2012,0,145,0,2013-01-24
4,4,2013-01/3,2013,0,0,54.60,20.70,10.10,8.30,5233.422,9673.9,12160.0,9480.422,30/04/2011,0,70,0,2013-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,556,2021-03/211435,2021,0,0,0.00,0.00,0.00,0.00,0.000,0.0,0.0,0.000,[],0,0,1,2021-03-31
557,557,2021-03/211852,2021,0,1,0.00,0.00,0.00,0.00,9828.216,18814.2,22860.0,18437.916,20/11/2019,0,140,1,2021-03-25
558,558,2021-03/211921,2021,0,1,0.00,0.00,0.00,0.00,8442.200,16980.0,20700.0,16640.400,16/01/2020,0,130,1,2021-03-29
559,559,2021-04/212089,2021,0,1,0.00,0.00,0.00,0.00,9610.870,18716.4,22800.0,18342.072,20/11/2019,0,140,1,2021-04-08


#### .- Paso 1.2.3 Sustituimos id de partida por  valor de partida en el dataframe con el que crearemos la siguiente tabla

In [15]:
df_InDt3 = df_InDt2.copy()
IdPart = zip(df_Ptda3["Partida"],df_Ptda3["IdPartida"])
PartID = dict(IdPart)
PartID

df_InDt3["Partida"].replace(to_replace=PartID, inplace=True)
df_InDt3["Partida"].unique()

array([ 47,  73,  74,  75,  48, 207, 211, 409,   0, 213, 225, 226, 229,
       230, 102, 103, 104,  49, 105, 515, 516, 517, 518, 519, 520, 521,
       522, 523, 524,  50,  51, 536, 543, 544, 548,   2, 106, 107,  52,
        53,   4, 108, 109,  54, 110, 111, 112, 113, 114,   5,   6,   7,
         8, 116,  55,  56, 117,  10,  57,  58,  59,  60, 118,  13, 119,
        14,  15,  16,  17, 125, 126,  63,  18,  64,  65,  66, 129,  19,
       131, 127, 128,  20, 214, 215, 216,  67,  68, 132,  71,  21,  22,
        23, 133, 134,  24,  25, 219,  72, 135,  76,  77,  78, 136, 137,
       139,  79, 141, 142,  80,  81, 143, 223,  82, 144, 533, 534, 535,
       539, 542, 558, 550, 551, 557, 559, 540, 531, 532, 537, 538, 541,
       545, 546, 547, 549, 552, 553, 554, 555, 556, 304, 322, 183, 403,
       404, 316, 405, 317,  26,  27,  28, 460, 205, 206,  29, 318,  30,
       461, 462,  83, 145, 208, 209, 210, 319, 406, 407, 320, 439,  87,
       408,  31,  32,  33,  97,  34, 463, 212, 464, 465, 146, 32

### .- Paso 1.3 Tabla pieza

In [16]:


# Creamos el dataframe con las columnas que corresponden a la tabla "pieza"
checkValues = [0,2,5,17,19,22,23,24]
colsPartidas = []
for pos in checkValues:
    colsPartidas.append(df_InDt3.columns[pos])

df_Pza1 = df_InDt3.loc[:,colsPartidas]

df_Pza1.insert(0,"IdPieza",df_Pza1.index,True)
df_Pza1["FechaSalida"] = df_Pza1["FechaSalida"].astype("str")
df_Pza1["PesoEntradaEntrada"] = df_Pza1["PesoEntradaEntrada"].fillna(-1)
df_Pza1["PesoSalida"] = df_Pza1["PesoSalida"].fillna(-1)

df_Pza1
# len(df_Pza1["IdPieza"])

,IdPieza,TipoArticuloEscandallo,Partida,NumeroSerieLc,PesoEntradaEntrada,PesoSalida,FechaSalida,TipoPesadaSalida,EstadoProductoSalida
0,0,0,47,140100102008,13.962100,9.400,2018-04-24 00:00:00.000,0,0
1,1,0,47,140100102030,14.580588,10.400,2018-06-04 00:00:00.000,0,1
2,2,1,47,141100102051,9.603106,5.964,2016-05-06 00:00:00.000,0,1
3,3,1,47,141100102001,7.682340,3.150,2016-12-01 00:00:00.000,1,1
4,4,1,47,141100102155,7.616813,3.054,2016-08-01 00:00:00.000,1,1
...,...,...,...,...,...,...,...,...,...
150429,150429,1,420,237030576292,10.149980,6.383,2021-04-13 00:00:00.000,0,1
150430,150430,1,420,237030576292,7.885824,3.160,2021-04-13 00:00:00.000,0,1
150431,150431,1,420,237030576292,10.149980,6.383,2021-04-16 00:00:00.000,1,1
150432,150432,1,420,237030576292,7.885824,3.160,2021-04-16 00:00:00.000,1,1


# PASO 2 CARGAR DATOS EN LA BBDD

 # Las tablas las llenaremos en el mismo orden que se crearon en la base de la siguiente manera:

## .- Paso 2.1 Cargar tablas de categoría

## .- Paso 2.2 Cargar tabla de partida
       
## .- Paso 2.3 Cargar tabla pieza

## .- Paso 2.4 Definir NULL Values

### .- Paso 2.1 Cargar tablas de categoría

In [17]:
# Funcion a la cual le introducimos el diccionario creado previamente y el nombre de la tabla correspondiente 
# en MySQL

def IntroTablasCategorias(diccionario, nombreColumna):
    import mysql.connector
    
    sqlcategoria = "INSERT INTO factoria_jamones."+str(nombreColumna)+"(id_"+str(nombreColumna)+", desc_"+str(nombreColumna)+") VALUES (%s, %s)"
    conex = mysql.connector.connect(
            host = "localhost",
            user = "root", 
            password = "",
            database ="factoria_jamones",
            port = 3306,
            autocommit=True)
    transaccion = conex.cursor()

    count = 0
    for values in diccionario:
        try:
            transaccion.execute(sqlcategoria, (diccionario[values], values))
        except: 
            print("elemento número "+ str(count+1) +" ya existe en base de datos")
            count+=1
    conex.close()

In [18]:
for cols in df_Base:
    if df_Base[cols].dtype == "int64":
        df_Base[cols]=df_Base[cols].astype("object")
        
df_Base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150434 entries, 0 to 150433
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   TipoArticuloEscandallo  150434 non-null  object 
 1   Añada                   150434 non-null  object 
 2   Partida                 150434 non-null  object 
 3   Raza                    150434 non-null  object 
 4   Alimentacion            150434 non-null  object 
 5   NumeroSerieLc           150434 non-null  object 
 6   AcidoOleico             150434 non-null  float64
 7   AcidoPalmitico          150434 non-null  float64
 8   AcidoEstearico          150434 non-null  float64
 9   AcidoLinoleico          150434 non-null  float64
 10  PesoVarios              150434 non-null  float64
 11  Pesovientres            150434 non-null  float64
 12  PesoCerdos              150434 non-null  float64
 13  PesoNetoCanal           150434 non-null  float64
 14  Nacimiento          

In [19]:
# Introducimos valores

IntroTablasCategorias(DiccCats(df_Base["Raza"].unique()),"raza")
IntroTablasCategorias(DiccCats(df_Base["CodigoMatadero"].unique()),"matadero")
IntroTablasCategorias(DiccCats(df_Base["Alimentacion"].unique()),"alimentacion")
IntroTablasCategorias(DiccCats(df_Base["TipoPesadaSalida"].unique()),"tipo_pesada_salida")
IntroTablasCategorias(DiccCats(df_Base["TipoPesada"].unique()),"tipo_pesada_entrada")
IntroTablasCategorias(DiccCats(df_Base["EstadoProductoSalida"].unique()),"estado_salida")
IntroTablasCategorias(DiccCats(df_Base["TipoArticuloEscandallo"].unique()),"tipo_pieza")

### .- Paso 2.2 Cargar tabla de partida

In [20]:
df_Ptda3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IdPartida       561 non-null    int64  
 1   Partida         561 non-null    object 
 2   Añada           561 non-null    int64  
 3   Raza            561 non-null    int64  
 4   Alimentacion    561 non-null    int64  
 5   AcidoOleico     561 non-null    float64
 6   AcidoPalmitico  561 non-null    float64
 7   AcidoEstearico  561 non-null    float64
 8   AcidoLinoleico  561 non-null    float64
 9   PesoVarios      561 non-null    float64
 10  Pesovientres    561 non-null    float64
 11  PesoCerdos      561 non-null    float64
 12  PesoNetoCanal   561 non-null    float64
 13  Nacimiento      561 non-null    object 
 14  CodigoMatadero  561 non-null    int64  
 15  NroAnimales     561 non-null    int64  
 16  TipoPesada      561 non-null    int64  
 17  FechaEntrada    561 non-null    obj

In [21]:
def IntroTablaPartida(df):
    import mysql.connector
        
    sqlcategoria = "INSERT INTO factoria_jamones.partida VALUES ("+(len(df.columns)-1)*"%s,"+"%s)"
    conex = mysql.connector.connect(
            host = "localhost",
            user = "root", 
            password = "",
            database ="factoria_jamones",
            port = 3306,
            autocommit=True)
    
    transaccion = conex.cursor()
    try:
        transaccion.executemany(sqlcategoria, df.values.tolist())
    except Exception as e: 
        print(e)
    conex.close()


In [22]:
IntroTablaPartida(df_Ptda3)

### .- Paso 2.3 Cargar tabla pieza

In [23]:
def IntroTablaPieza(df, IdColumn):
    import mysql.connector
    
    sqlcategoria = "INSERT INTO factoria_jamones.pieza VALUES ("+(len(df.columns)-1)*"%s,"+"%s)"
    conex = mysql.connector.connect(
            host = "localhost",
            user = "root", 
            password = "",
            database ="factoria_jamones",
            port = 3306,
            autocommit=True)
    
    
    dicc = df.set_index(IdColumn).T.to_dict('list')
    count = 0
    
    for keys in dicc:        
        transaccion = conex.cursor()
        try:
            transaccion.execute(sqlcategoria, (keys, dicc[keys][0],dicc[keys][1],dicc[keys][2],dicc[keys][3],dicc[keys][4],dicc[keys][5],dicc[keys][6],dicc[keys][7]))
        except Exception as e: 
            print(e)
        count+=1
    conex.close()

In [24]:
IntroTablaPieza(df_Pza1,"IdPieza")

In [25]:
df_Ptda3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IdPartida       561 non-null    int64  
 1   Partida         561 non-null    object 
 2   Añada           561 non-null    int64  
 3   Raza            561 non-null    int64  
 4   Alimentacion    561 non-null    int64  
 5   AcidoOleico     561 non-null    float64
 6   AcidoPalmitico  561 non-null    float64
 7   AcidoEstearico  561 non-null    float64
 8   AcidoLinoleico  561 non-null    float64
 9   PesoVarios      561 non-null    float64
 10  Pesovientres    561 non-null    float64
 11  PesoCerdos      561 non-null    float64
 12  PesoNetoCanal   561 non-null    float64
 13  Nacimiento      561 non-null    object 
 14  CodigoMatadero  561 non-null    int64  
 15  NroAnimales     561 non-null    int64  
 16  TipoPesada      561 non-null    int64  
 17  FechaEntrada    561 non-null    obj

### .- Paso 2.4 Definir NULL Values en MySQL

In [26]:
def NullValuesBBDD(tabla, ListaColumnas):
    import mysql.connector

    conex = mysql.connector.connect(
            host = "localhost",
            user = "root", 
            password = "",
            database ="factoria_jamones",
            port = 3306,
            autocommit=True)

    for columna in ListaColumnas:
        sqlNullValuesBBDD = "UPDATE "+ str(tabla)+" SET "+str(tabla)+"."+str(columna)+" = NULL WHERE "+str(tabla)+"."+str(columna)+"= "+str("-1")
        transaccion = conex.cursor()
        try:
            transaccion.execute(sqlNullValuesBBDD)
        except Exception as e: 
            print(e)
    conex.close()

In [27]:
ListaColumnasPieza = ["peso_entrada",
                      "peso_salida"]

ListaColumnasPartida = ["acido_oleico",
                        "acido_palmitico",
                        "acido_estearico",
                        "acido_linoleico",
                        "peso_varios",
                        "peso_vientres", 
                        "peso_cerdos", 
                        "peso_neto_canal",
                        ]
ListaColumnasMatadero = ["desc_matadero"]



ListaTablas = ["pieza", "partida", "matadero"]

dicc = {"pieza":ListaColumnasPieza,"partida":ListaColumnasPartida,"matadero":ListaColumnasMatadero}


In [28]:
for tablas in dicc:
    NullValuesBBDD(tablas, dicc[tablas])